<a href="https://colab.research.google.com/github/scobin/colab-tutorial/blob/main/gspread.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Purpose
This article will show some simple usages of **gspread** library.  
- check & update version.
- read the google spreadsheet in your google drive by gspread.
- read workbook, sheet, cell by gspread.
- update cell value. 


## Mount google drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## gspread
use gspread to deal with google spreadsheet.

### old version of gspread is already in colab.

In [ ]:
import gspread
print(gspread.__version__)

4.0.1


### Install new version of gspread

In [ ]:
!pip install --upgrade gspread
from importlib import reload
reload(gspread)
print(gspread.__version__)
# => 4.0.1

4.0.1


## Authenticate of Google Spreadsheet

In [ ]:
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

## Usage of gspread

In [ ]:
import pandas as pd

Read spreadsheet by name, url, key.

In [ ]:
workbook = gc.open_by_key("14WGIODpkL485LM2-RQfJI53VONxrslAJuLxlQhy9yQg")

In [ ]:
# workbook = gc.open('test')
# workbook = gc.open_by_url('https://docs.google.com/spreadsheets/d/14WGIODpkL485LM2-RQfJI53VONxrslAJuLxlQhy9yQg/')
workbook

<Spreadsheet 'test' id:14WGIODpkL485LM2-RQfJI53VONxrslAJuLxlQhy9yQg>

get *all worksheets*  


In [ ]:
sheets = workbook.worksheets()
sheets

APIError: ignored

get *worksheet* by name, index, id.

In [ ]:
sheet = workbook.worksheet('data1')
# sheet = workbook.get_worksheet(0)
# sheet = workbook.get_worksheet_by_id(674349371)
sheet

<Worksheet 'data1' id:0>

### get cell's value.
- get all value as dict objects by using `get_all_records`.
- get all value as list objects by using `get_all_values`.
- get value of specific range by using `get_values`.
- use pandas to show the values.

In [ ]:
sheet.get_all_records()

[{'age': 14, 'id': 1, 'name': 'John', 'no': 2101},
 {'age': 14, 'id': 2, 'name': 'Tom', 'no': 2102},
 {'age': 15, 'id': 3, 'name': 'Andy', 'no': 2103},
 {'age': 15, 'id': 4, 'name': 'Mary', 'no': 2104},
 {'age': 14, 'id': 5, 'name': 'Lisa', 'no': 2105}]

In [ ]:
sheet.get_all_values()

[['id', 'no', 'name', 'age'],
 ['1', '2101', 'John', '14'],
 ['2', '2102', 'Tom', '14'],
 ['3', '2103', 'Andy', '15'],
 ['4', '2104', 'Mary', '15'],
 ['5', '2105', 'Lisa', '14']]

In [ ]:
sheet.get_values('A:C')

[['id', 'no', 'name'],
 ['1', '2107', 'John'],
 ['2', '2102', 'Tom'],
 ['3', '2103', 'Andy'],
 ['4', '2104', 'Mary'],
 ['5', '2105', 'Lisa']]

In [ ]:
values = sheet.get_all_values()
df = pd.DataFrame(values[1:], columns=values[0])
df

,id,no,name,age
0,1,2107,John,16
1,2,2102,Tom,16
2,3,2103,Andy,16
3,4,2104,Mary,16
4,5,2105,Lisa,16
5,,,,80


In [ ]:
records = sheet.get_all_records()
df = pd.DataFrame(records)
df

,id,no,name,age
0,1,2107,John,16
1,2,2102,Tom,16
2,3,2103,Andy,16
3,4,2104,Mary,16
4,5,2105,Lisa,16
5,,,,80


### Update cell's value
- update range's value by using `update` or `update_cells`.    
- update one cell's value by using `update_cell`.  

In [ ]:
sheet.update('D2:D6', [[16], [16], [16], [16], [16]])
sheet.update('D7',  '=SUM(D2:D6)', raw=False)
sheet.get_all_values()

[['id', 'no', 'name', 'age'],
 ['1', '2107', 'John', '16'],
 ['2', '2102', 'Tom', '16'],
 ['3', '2103', 'Andy', '16'],
 ['4', '2104', 'Mary', '16'],
 ['5', '2105', 'Lisa', '16'],
 ['', '', '', '80']]

In [ ]:
sheet.update_cell(2, 2, 2107)
sheet.get_all_values()

{'spreadsheetId': '14WGIODpkL485LM2-RQfJI53VONxrslAJuLxlQhy9yQg',
 'updatedCells': 1,
 'updatedColumns': 1,
 'updatedRange': 'data1!B2',
 'updatedRows': 1}

In [ ]:
age_list = sheet.range('D2:D6')
age_list

[<Cell R2C4 '14'>,
 <Cell R3C4 '14'>,
 <Cell R4C4 '15'>,
 <Cell R5C4 '15'>,
 <Cell R6C4 '14'>]

In [ ]:
for age in age_list:
  age.value = 14

sheet.update_cells(age_list)
sheet.get_all_values()

[['id', 'no', 'name', 'age'],
 ['1', '2107', 'John', '14'],
 ['2', '2102', 'Tom', '14'],
 ['3', '2103', 'Andy', '14'],
 ['4', '2104', 'Mary', '14'],
 ['5', '2105', 'Lisa', '14']]